<h1>World Factbook</h1>

In this guided project, we'll use SQL to explore and analyze demographic data from the <a href="https://www.cia.gov/library/publications/the-world-factbook/">CIA World Factbook</a> database.

In [221]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [222]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


This database contains two tables: sqlite_sequence and facts, but we are only interested in the second table since it contains facts about countries. 

Let's return the first ten rows of the facts table.

In [223]:
%%sql
SELECT *
  FROM facts
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


Let's see the top 10 largest populations, calculate some summary statistics and look for any outlier countries by using min and max functions.

In [286]:
%%sql
SELECT name, population/1000000 'pop (M)'
FROM facts 
ORDER BY population DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,pop (M)
World,7256
China,1367
India,1251
European Union,513
United States,321
Indonesia,255
Brazil,204
Pakistan,199
Nigeria,181
Bangladesh,168


In [285]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population)/1000000 AS 'max_pop (M)',
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop (M),min_pop_growth,max_pop_growth
0,7256,0.0,4.02


A few things stick out from the summary statistics:

    There's a country with a population of 0
    There's a country with a population of more than 7.2 billion people

Let's use subqueries to zoom in on just these outlier countries without using the specific values.

In [226]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It seems like the table contains a row for Antarctica, which explains the population of 0.

In [383]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None




We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.


In [386]:
%%sql
SELECT MIN(population) min_pop, MAX(population) max_pop,
       MIN(population_growth)min_pop_growth, MAX(population_growth)max_pop_growth
  FROM facts
 WHERE name NOT IN ("World");

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [377]:
%%sql
SELECT CAST(ROUND(AVG(population)/1000000,0)AS INT) AS 'avg_population(M)', 
ROUND(AVG(area)/1000000,2) AS 'avg_area(M)'
  FROM facts
 WHERE name <> 'World'AND name <> "European Union";



 * sqlite:///factbook.db
Done.


avg_population(M),avg_area(M)
30,0.54


In [388]:
%%sql
SELECT CAST(ROUND(AVG(area)/1000,2)AS INT) 'avg_country_area (Sq. km, 1000s)'
  FROM facts
 WHERE name <> "European Union";

 * sqlite:///factbook.db
Done.


"avg_country_area (Sq. km, 1000s)"
539


We see that the average population is 30 million and the average land area is circa half a million square kilometers. <br>The EU and world had to be excluded from the population computations, since both already have their countries in the dataset.

Now we'll show the countries with the highest population densities.<br>
The area_land column is - the country's land area in square kilometers.

In [312]:
%%sql
SELECT name, population/area_land 'Population Density (Inhabitants per Square Km)',
       ROUND(CAST(population AS FLOAT)/1000000,2) 'pop (M)',area_land 'Land Area (Square Km)'
FROM facts
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,Population Density (Inhabitants per Square Km),pop (M),Land Area (Square Km)
Macau,21168,0.59,28
Monaco,15267,0.03,2
Singapore,8259,5.67,687
Hong Kong,6655,7.14,1073
Gaza Strip,5191,1.87,360
Gibraltar,4876,0.03,6
Bahrain,1771,1.35,760
Maldives,1319,0.39,298
Malta,1310,0.41,316
Bermuda,1299,0.07,54


In [303]:
%%sql
SELECT CAST(ROUND(AVG((population) / area_land), 0)AS INT) 'Average Population Density'
  FROM facts
 WHERE name <> 'Antarctica'



 * sqlite:///factbook.db
Done.


Average Population Density
431


The top five most densely populated countries are over ten times more densely populated than the average 'country'. <br>Most are considered city states so this might be expected.
<p>Now let's see countries with above average population, but below average land area.

In [661]:
%%sql
SELECT
name,area/1000 'area(K)',population/1000000 'pop.(M)',(SELECT CAST(AVG(population)/1000000 AS INT) FROM facts) 'average pop(M)'
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts)
      AND
      area < (SELECT AVG(area)
              FROM facts);

 * sqlite:///factbook.db
Done.


name,area(K),pop.(M),average pop(M)
Bangladesh,148,168,62
Germany,357,80,62
Japan,377,126,62
Philippines,300,100,62
Thailand,513,67,62
United Kingdom,243,64,62
Vietnam,331,94,62


<h2>Demographic Trends</h2>
<h3>Country Comparison : Population Growth Rate</h3><br>
Population growth rate compares the average annual percent change in populations, <br>resulting from a surplus (or deficit) of births over deaths and the balance of migrants entering and leaving a country. <br>The rate may be positive or negative. <br>
Below compares for countries with >= 30M populations.

In [622]:
%%sql
SELECT RANK() OVER(ORDER BY population_growth DESC ) as 'rank',
            name as Country,population/1000000 'pop(M)',birth_rate,death_rate, population_growth,
            (SELECT ROUND(AVG(population_growth),2) FROM facts
            WHERE name <>"European Union"
            AND name <>"World")'world average pop.growth'
FROM facts
WHERE population>=30000000 AND population_growth >(SELECT AVG(population_growth)
                                                    FROM facts WHERE name <>"European Union"
                                                  AND name <>"World")
ORDER BY population_growth DESC
LIMIT 15;

 * sqlite:///factbook.db
Done.


rank,Country,pop(M),birth_rate,death_rate,population_growth,world average pop.growth
1,Uganda,37,43.79,10.69,3.24,1.21
2,Iraq,37,31.45,3.77,2.93,1.21
3,Ethiopia,99,37.27,8.19,2.89,1.21
4,Tanzania,51,36.39,8.0,2.79,1.21
5,"Congo, Democratic Republic of the",79,34.88,10.07,2.45,1.21
5,Nigeria,181,37.64,12.9,2.45,1.21
7,Afghanistan,32,38.57,13.89,2.32,1.21
8,Kenya,45,26.4,6.89,1.93,1.21
9,Algeria,39,23.67,4.31,1.84,1.21
10,Egypt,88,22.9,4.77,1.79,1.21


In [599]:
%%sql
SELECT AVG(population_growth)'EU population growth rate'
FROM facts WHERE name ="European Union"

 * sqlite:///factbook.db
Done.


EU population growth rate
0.25


We see that it is mainly African countries that have the highest population growth.<br>This coincides with this <a href="https://twitter.com/aronstrandberg/status/1147867848104194048" target="_blank"> data visualisation</a> and research here : <a href="https://www.pewresearch.org/fact-tank/2019/07/10/for-world-population-day-a-look-at-the-countries-with-the-biggest-projected-gains-and-losses-by-2100/" target="_blank">Pew Research Center</a> that shows Africa is expected to gain the most people by the end of the century in 2100, so we can be confident in our results. <p>The European Union (EU) is almost 5 times as small as the average : 0.25 vs 1.21. <p>European countries (including non-EU ones) have largely fertility rates below replacement levels and the European population is expected to peak in 2040.<br>Europe as a region in 2019 had the highest percentage of people 65+.
    
<h3>Field Listing :: Birth rate</h3>
This entry gives the average annual number of births during a year per 1,000 persons in the population at midyear.<br> The birth rate is usually the dominant factor in determining the rate of population growth. <br>It depends on both the level of fertility and the age structure of the population. 

In [623]:
%%sql
SELECT name,population/1000000 'pop.(M)',birth_rate,death_rate,population_growth
FROM facts WHERE population>1000000 AND name in ('European Union','Russia','Poland','Romania','France','Italy','Spain',
                          'Germany','United Kingdom','Sweden',
                         'Andorra','Armenia','Azerbaijan','Belarus','Georgia','Iceland')
order by 2 desc


 * sqlite:///factbook.db
Done.


name,pop.(M),birth_rate,death_rate,population_growth
European Union,513,10.2,10.2,0.25
Russia,142,11.6,13.69,0.04
Germany,80,8.47,11.42,0.17
France,66,12.38,9.16,0.43
United Kingdom,64,12.17,9.35,0.54
Italy,61,8.74,10.19,0.27
Spain,48,9.64,9.04,0.89
Poland,38,9.74,10.19,0.09
Romania,21,9.14,11.9,0.3
Azerbaijan,9,16.64,7.07,0.96


We see that as the birth_rate is in close range of the death rate, European populations are either very slowly increasing  or decreasing. Low fertility, as represented by the birth_rate (births per 1000), is a main factor in accelerating the ageing of European populations.

In [321]:
%%sql
SELECT name, 
ROUND(CAST(population AS FLOAT)/
       (SELECT population FROM facts WHERE name = 'China')*100,1)||'%' 
       'population as percent of China'
FROM facts WHERE name = 'France';



 * sqlite:///factbook.db
Done.


name,population as percent of China
France,4.9%


In [340]:
%%sql
SELECT name,CAST(CAST(Population AS FLOAT) / 
(SELECT MAX(population) FROM facts)*100 AS INT)||'%' 'Percent of world population'
FROM facts
WHERE name == (SELECT name
               FROM facts
               WHERE population == (SELECT MAX(population)
                                    FROM facts
                                    WHERE name <> 'World'));

 * sqlite:///factbook.db
Done.


name,Percent of world population
China,18%


In [482]:
%%sql
SELECT RANK() OVER(ORDER BY population DESC ) as Ranking, name,population,
CAST(CAST(population AS FLOAT) / 
(SELECT MAX(population) FROM facts)*100 AS INT)||'%' 'Percent of world population'
FROM facts
WHERE name <> 'World' AND name <> 'European Union'
ORDER BY 3 DESC
LIMIT 5;




 * sqlite:///factbook.db
Done.


Ranking,name,population,Percent of world population
1,China,1367485388,18%
2,India,1251695584,17%
3,United States,321368864,4%
4,Indonesia,255993674,3%
5,Brazil,204259812,2%


In [440]:
%%sql
SELECT name 'countries with >2x area of India', area/1000000 'area (M)',
(SELECT area/1000000 FROM facts
  WHERE name = 'India')AS 'India area (M)'
FROM facts
  WHERE area > 
    (SELECT 2*area FROM facts WHERE name = 'India')

 * sqlite:///factbook.db
Done.


countries with >2x area of India,area (M),India area (M)
Australia,7,3
Brazil,8,3
Canada,9,3
China,9,3
Russia,17,3
United States,9,3


birth_rate - The country's birth rate, or the number of births a year per 1,000 people.

In [516]:
%%sql
WITH top10births AS
    (
    SELECT birth_rate/ (SELECT SUM(birth_rate) FROM facts) f
      FROM facts
     ORDER BY 1 DESC
     LIMIT 10
    )
    
    SELECT ROUND(SUM(f) * 100, 2) 'top10 countries by birth rates (%) of total'
  FROM top10births

 * sqlite:///factbook.db
Done.


top10 countries by birth rates (%) of total
9.53


In [645]:
    %%sql
    SELECT name,birth_rate,
    (select cast(SUM(birth_rate)as int) FROM facts) 'world birth_rate sum'  
      FROM facts
     ORDER BY 2 DESC
     LIMIT 10

 * sqlite:///factbook.db
Done.


name,birth_rate,world birth_rate sum
Niger,45.45,4406
Mali,44.99,4406
Uganda,43.79,4406
Zambia,42.13,4406
Burkina Faso,42.03,4406
Burundi,42.01,4406
Malawi,41.56,4406
Somalia,40.45,4406
Angola,38.78,4406
Mozambique,38.58,4406


<h3> Show countries with pop > Brazil but less than China </h3>

In [244]:
%%sql
SELECT name, population/1000000 'pop (M)'
FROM facts 
WHERE population > (SELECT population FROM facts WHERE name = 'Brazil') 
AND population < (SELECT population FROM facts WHERE name = 'China')
AND NOT name = 'European Union'


 * sqlite:///factbook.db
Done.


name,pop (M)
India,1251
Indonesia,255
United States,321


 <h3>Field Listing :: Population growth rate</h3>
The average annual percent change in the population, resulting from a surplus (or deficit) of births over deaths and the balance of migrants entering and leaving a country. The rate may be positive or negative. The growth rate is a factor in determining how great a burden would be imposed on a country by the changing needs of its people for infrastructure (e.g., schools, hospitals, housing, roads), resources (e.g., food, water, electricity), and jobs. Rapid population growth can be seen as threatening by neighboring countries. </p>

The population_growth column is the country's population growth as a percentage and so predicted increase in millions of people is derived as below.

In [646]:
%%sql
SELECT name, population/1000000 'pop (M)', population_growth 'population_growth%', 
CAST((population * (population_growth/ 100 + 1) - population)/1000000 AS INT) 'predicted increase (M)'
FROM facts
ORDER BY 4 DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,pop (M),population_growth%,predicted increase (M)
World,7256,1.08,78
India,1251,1.22,15
China,1367,0.45,6
Nigeria,181,2.45,4
Bangladesh,168,1.6,2
Ethiopia,99,2.89,2
Indonesia,255,0.92,2
Pakistan,199,1.46,2
United States,321,0.78,2
Brazil,204,0.77,1
